#### 1. References 

This notebook heavily uses concepts and implementation of:
* https://github.com/JingweiToo/Wrapper-Feature-Selection-Toolbox-Python

In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', size=16) 
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
import warnings
import logging
from sklearn.model_selection import train_test_split


import numpy as np
from numpy.random import rand
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
from random import randrange
import time

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
x_train = np.load("/kaggle/input/homework2timeseries/x_train.npy")
y_train = np.load("/kaggle/input/homework2timeseries/y_train.npy")

In [ ]:
print("X_train Shape:", x_train.shape)
print("Y_train Shape:", y_train.shape)

In [ ]:
#x_train = x_train[0:, 0:25, :]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.1, random_state=seed, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=seed, shuffle=True)

In [ ]:
y_train = tfk.utils.to_categorical(y_train)
y_test = tfk.utils.to_categorical(y_test)
y_val = tfk.utils.to_categorical(y_val)

X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape

In [ ]:
input_shape = X_train.shape[1:]

classes = y_train.shape[-1]
batch_size = 16
epochs = 50

In [ ]:
input_shape

#### Vanilla Long Short Term Memory (LSTM) Neural Network

In [ ]:
def build_LSTM_classifier(input_shape, classes):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # Feature extractor
    lstm = tfkl.Bidirectional(tfkl.LSTM(256, return_sequences=True))(input_layer)
    lstm = tfkl.Bidirectional(tfkl.LSTM(512, return_sequences=True))(lstm)
    Skip_LSTM = tfkl.LSTM(256, return_sequences=True)(input_layer)
    #Skip_LSTM = tfkl.LSTM(128, return_sequences=True)(Skip_LSTM)

    lstm = tfkl.GlobalMaxPooling1D()(lstm)
    Skip_LSTM = tfkl.GlobalMaxPooling1D()(Skip_LSTM)
    dropout = tfkl.Dropout(.3, seed=seed)(lstm)
    Skip_LSTM = tfkl.Dropout(.3)(Skip_LSTM)
    flatt=tfkl.concatenate([dropout,Skip_LSTM])
    # Classifier
    classifier = tfkl.Dense(256, activation='relu')(flatt)
   # classifier = tfkl.Dense(64, activation='relu')(classifier)
    output_layer = tfkl.Dense(classes, activation='softmax')(classifier)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [ ]:
model_LSTM = build_LSTM_classifier(input_shape, classes)
model_LSTM.summary()

In [ ]:
# error rate
def error_rate(xtrain, ytrain, x, opts):
    # parameters
    k  = opts['k']
    fold = opts['fold']
    xt = fold['xt']
    yt = fold['yt']
    xv = fold['xv']
    yv = fold['yv']
    
    # number of instances
    num_train = np.size(xt, 0)
    num_valid = np.size(xv, 0)
    # Define selected features
    xtrain = xt[:, x == 1, :]
    xvalid = xv[:, x == 1, :]
    # Training
    input_shape = xtrain.shape[1:]
    model_LSTM = build_LSTM_classifier(input_shape, 12)
    # Train the model
    history_LSTM = model_LSTM.fit(
        [xtrain],
        y = yt,
        batch_size = 16,
        epochs = 50,
        validation_data=(xvalid, yv),
        callbacks = [
            tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
            tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', patience = 3, verbose=1,factor=0.3, min_lr=0.000001),
            tfk.callbacks.ModelCheckpoint('LSTM.hdf5', save_best_only=True, monitor='val_loss', mode='min')

        ]
    ).history
    # Prediction
    ypred   = model_LSTM.predict(xvalid)
    acc     = np.sum(yv == ypred) / num_valid
    error   = 1 - acc
    
    return error, acc

In [ ]:
# Error rate & Feature size
def Fun(xtrain, ytrain, x, opts):
    # parameters
    alpha = 0.99
    beta = 1 - alpha
    # original feature size
    max_feat = len(x)
    # Number of selected features
    num_feat = np.sum(x == 1)
    # Solve if no feature selected
    if num_feat == 0:
        cost = 1
    else:
        # Get error rate
        error, acc = error_rate(xtrain, ytrain, x, opts)
        # Objective function
        cost = alpha * error + beta * (num_feat / max_feat)
        
    return cost

In [ ]:
def init_position(lb, ub, N1, N2, dim):
    X = np.zeros([N1, N2, dim], dtype='float')
    for i in range(N1):
        for j in range(N2):
                X[i,j,:] = lb[0,j,:] + (ub[0,j,:] - lb[0,j,:]) * rand()        
    
    return X

In [ ]:
def binary_conversion(X, thres, N1, N2, dim):
    Xbin = np.zeros([N1, N2, dim], dtype='int')
    for i in range(N1):
        for j in range(N2):
                if X[i,j,:] > thres:
                    Xbin[i,j,:] = 1
                else:
                    Xbin[i,j,:] = 0
    
    return Xbin

In [ ]:
def roulette_wheel(prob):
    num = len(prob)
    C   = np.cumsum(prob)
    P   = rand()
    for i in range(num):
        if C[i] > P:
            index = i;
            break
    
    return index

In [ ]:
def jfs(xtrain, ytrain, opts):
    # Parameters
    ub = 1
    lb = 0
    thres = 0.5
    CR = 0.8  # crossover rate
    MR = 0.01 # mutation rate
    
    N1 = opts['N']
    N2 = 36
    max_iter = opts['T']
    if 'CR' in opts:
        CR = opts['CR']
    if 'MR' in opts:
        MR = opts['MR']
    
    # Dimension
    dim = np.size(xtrain, 2)
    if np.size(lb) == 1:
        ub = ub * np.ones([1, N2, dim], dtype='float')
        lb = lb * np.ones([1, N2, dim], dtype='float')
    
    # Initialize position
    X = init_position(lb, ub, N1, N2, dim)
    
    # Binary conversion
    X = binary_conversion(X, thres, N1, N2, dim)
    
    # Fitness at first iteration
    fit = np.zeros([N, 1], dtype='float')
    Xgb = np.zeros([1, N2, dim], dtype='int')
    fitG = float('inf')
    
    for i in range(N1):
        fit[i, 0] = Fun(xtrain, ytrain, X[i, :, :], opts)
        if fit[i, 0] < fitG:
            Xgb[0, :, :] = X[i, :, :]
            fitG = fit[i,0]
    
    # Pre
    curve = np.zeros([1, max_iter], dtype='float')
    t     = 0
    
    curve[0,t] = fitG.copy()
    print("Generation:", t + 1)
    print("Best (GA):", curve[0,t])
    t += 1
    
    while t < max_iter:
        # Probability
        inv_fit = 1 / (1 + fit)
        prob = inv_fit / np.sum(inv_fit)
        # Number of crossovers
        Nc = 0
        for i in range(N1):
            if rand() < CR:
                Nc += 1
        
        x1 = np.zeros([Nc, N2, dim], dtype='int')
        x2 = np.zeros([Nc, N2, dim], dtype='int')
        for i in range(Nc):
            # Parent selection
            k1  = roulette_wheel(prob)
            k2  = roulette_wheel(prob)
            P1 = X[k1, :, :].copy()
            P2 = X[k2, :, :].copy()
            # Random one dimension from 1 to dim
            index = np.random.randint(low=1, high=dim-1)
            # Crossover
            x1[i, :, :] = np.concatenate((P1[0:index], P2[index:]))
            x2[i, :, :] = np.concatenate((P2[0:index], P1[index:]))
            # Mutation
            for j in range(N2):
                if rand() < MR:
                    x1[i,j,:] = 1 - x1[i,j,:]

                if rand() < MR:
                    x2[i,j,:] = 1 - x2[i,j,:]
        # Merge two group into one
        Xnew = np.concatenate((x1, x2), axis=0)
            
        # Fitness
        Fnew = np.zeros([2 * Nc, 1], dtype='float')
        for i in range(2 * Nc):
            Fnew[i,0] = Fun(xtrain, ytrain, Xnew[i,:, :], opts)
            if Fnew[i,0] < fitG:
                Xgb[0,j,:] = Xnew[i,:,:]
                fitG = Fnew[i,0]
            
        # Store result
        curve[0,t] = fitG.copy()
        print("Generation:", t + 1)
        print("Best (GA):", curve[0,t])
        t += 1
            
        # Elitism 
        XX  = np.concatenate((X , Xnew), axis=0)
        FF  = np.concatenate((fit , Fnew), axis=0)
        # Sort in ascending order
        ind = np.argsort(FF, axis=0)
        for i in range(N1):
            X[i,:]   = XX[ind[i,0],:,:]
            fit[i,0] = FF[ind[i,0]]

    # Best feature subset
    Gbin       = Xgb[0,:,:]
    Gbin       = Gbin.reshape(dim)
    pos        = np.asarray(range(0, dim))    
    sel_index  = pos[Gbin == 1]
    num_feat   = len(sel_index)
    # Create dictionary
    ga_data = {'sf': sel_index, 'c': curve, 'nf': num_feat}
    
    return ga_data

In [ ]:
fold = {'xt':X_train, 'yt':y_train, 'xv':X_val, 'yv':y_val}

In [ ]:
print(y_val.shape)

In [ ]:
P    = 0.8       # switch probability
k     = 5     # k-value in KNN
N     = 20    # number of
T     = 10   # maximum number of iterations
opts = {'k':k, 'fold':fold, 'N':N, 'T':T, 'P':P}

In [ ]:
# perform feature selection
start_time = time.time()
fmdl  = jfs(X_train, y_train, opts)
print("Run Time --- %s seconds ---" % (time.time() - start_time))

sf    = fmdl['sf']

# model with selected features
x_train   = trainX_f[:, :,  sf]
x_tes   = testX_f[:, :, sf]
x_val = valX_f[:, :, sf]

num_test = np.size(x_test, 0)
    
# Training
model_LSTM = build_LSTM_classifier(input_shape, classes)
# Train the model
history_LSTM = model_LSTM.fit(
        [x_train],
        y = ytrain,
        batch_size = 16,
        epochs = 50,
        validation_data=(x_val, yvalid),
        callbacks = [
            tfk.callbacks.EarlyStopping(monitor='val_loss', mode='max', patience=10, restore_best_weights=True),
            tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', patience = 3, verbose=1,factor=0.3, min_lr=0.000001),
            tfk.callbacks.ModelCheckpoint('LSTM.hdf5', save_best_only=True, monitor='val_loss', mode='min')

        ]
    ).history
# Prediction
ypred   = model_LSTM.predict(x_test)
acc     = np.sum(y_test == ypred) / num_test

# Prediction


print("Accuracy:", acc)

# number of selected features
num_feat = fmdl['nf']
print("Feature Size:", num_feat)

# plot convergence
curve   = fmdl['c']
curve   = curve.reshape(np.size(curve,1))
x       = np.arange(0, opts['T'], 1.0) + 1.0

fig, ax = plt.subplots()
ax.plot(x, curve, 'o-')
ax.set_xlabel('Number of Iterations')
ax.set_ylabel('Fitness')
ax.set_title('Genetic')
ax.grid()
plt.show()